# kNN для произвольных метрик близости

### Сколько очков наберет игрок в сезоне?

In [1]:
import pandas as pd
import numpy as np

- player – имя игрока
- pos – роль игрока на поле
- g – количество матчей, в которых участвовал игрок
- gs – количество матчей, в которых игрок был на поле с самого начала
- pts – общее количество очков

pos:
- SF small forward
- C center
- PF power forward
- SG shooting guard
- PG point guard
- G guard
- F forward

In [2]:
data = pd.read_csv( 'nba_2013.csv' )
data.head()

,player,pos,age,bref_team_id,g,gs,mp,fg,fga,fg.,...,drb,trb,ast,stl,blk,tov,pf,pts,season,season_end
0,Quincy Acy,SF,23,TOT,63,0,847,66,141,0.468,...,144,216,28,23,26,30,122,171,2013-2014,2013
1,Steven Adams,C,20,OKC,81,20,1197,93,185,0.503,...,190,332,43,40,57,71,203,265,2013-2014,2013
2,Jeff Adrien,PF,27,TOT,53,12,961,143,275,0.520,...,204,306,38,24,36,39,108,362,2013-2014,2013
3,Arron Afflalo,SG,28,ORL,73,73,2552,464,1011,0.459,...,230,262,248,35,3,146,136,1330,2013-2014,2013
4,Alexis Ajinca,C,25,NOP,56,30,951,136,249,0.546,...,183,277,40,23,46,63,187,328,2013-2014,2013


In [16]:
data.columns

Index(['player', 'pos', 'age', 'bref_team_id', 'g', 'gs', 'mp', 'fg', 'fga',
       'fg.', 'x3p', 'x3pa', 'x3p.', 'x2p', 'x2pa', 'x2p.', 'efg.', 'ft',
       'fta', 'ft.', 'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf',
       'pts', 'season', 'season_end'],
      dtype='object')

Для построения модели возьмем несколько столбцов

In [148]:
predictionColumns = ['pos', 'age', 'g', 'gs', 'mp', 'fg', 'fga', 'fg.',  'x3p', 'x3pa', 'efg.', 'ft',
       'fta', 'ft.', 'orb']

In [149]:
dataForPrediction = data[ predictionColumns ]
dataForPrediction.head()

,pos,age,g,gs,mp,fg,fga,fg.,x3p,x3pa,efg.,ft,fta,ft.,orb
0,SF,23,63,0,847,66,141,0.468,4,15,0.482,35,53,0.660,72
1,C,20,81,20,1197,93,185,0.503,0,0,0.503,79,136,0.581,142
2,PF,27,53,12,961,143,275,0.520,0,0,0.520,76,119,0.639,102
3,SG,28,73,73,2552,464,1011,0.459,128,300,0.522,274,336,0.815,32
4,C,25,56,30,951,136,249,0.546,0,1,0.546,56,67,0.836,94


pos - категориальная переменная, надо перевести в количественную

In [150]:
dataForPrediction = pd.get_dummies( dataForPrediction )
dataForPrediction.head()

,age,g,gs,mp,fg,fga,fg.,x3p,x3pa,efg.,...,fta,ft.,orb,pos_C,pos_F,pos_G,pos_PF,pos_PG,pos_SF,pos_SG
0,23,63,0,847,66,141,0.468,4,15,0.482,...,53,0.660,72,0,0,0,0,0,1,0
1,20,81,20,1197,93,185,0.503,0,0,0.503,...,136,0.581,142,1,0,0,0,0,0,0
2,27,53,12,961,143,275,0.520,0,0,0.520,...,119,0.639,102,0,0,0,1,0,0,0
3,28,73,73,2552,464,1011,0.459,128,300,0.522,...,336,0.815,32,0,0,0,0,0,0,1
4,25,56,30,951,136,249,0.546,0,1,0.546,...,67,0.836,94,1,0,0,0,0,0,0


Названия колонок нашего dataframe изменились

In [151]:
predictionColumns = dataForPrediction.columns.values
predictionColumns

array(['age', 'g', 'gs', 'mp', 'fg', 'fga', 'fg.', 'x3p', 'x3pa', 'efg.',
       'ft', 'fta', 'ft.', 'orb', 'pos_C', 'pos_F', 'pos_G', 'pos_PF',
       'pos_PG', 'pos_SF', 'pos_SG'], dtype=object)

Проведем нормализацию данных

In [152]:
dataNormalized = ( dataForPrediction - dataForPrediction.mean() ) / dataForPrediction.std()
dataNormalized.head()

,age,g,gs,mp,fg,fga,fg.,x3p,x3pa,efg.,...,fta,ft.,orb,pos_C,pos_F,pos_G,pos_PF,pos_PG,pos_SF,pos_SG
0,-0.835906,0.384886,-0.862207,-0.435088,-0.738401,-0.768505,0.319884,-0.700282,-0.716608,0.012541,...,-0.515408,-0.389712,0.260690,-0.479271,-0.045596,-0.045596,-0.498831,-0.462818,1.962285,-0.540742
1,-1.550487,1.095711,-0.187863,-0.045011,-0.581271,-0.649215,0.674593,-0.778936,-0.829601,0.223485,...,0.117019,-0.882950,1.387883,2.082165,-0.045596,-0.045596,-0.498831,-0.462818,-0.508550,-0.540742
2,0.116868,-0.010016,-0.457600,-0.308035,-0.290291,-0.405214,0.846880,-0.778936,-0.829601,0.394250,...,-0.012515,-0.520826,0.743773,-0.479271,-0.045596,-0.045596,2.000520,-0.462818,-0.508550,-0.540742
3,0.355062,0.779789,1.599148,1.465144,1.577804,1.590172,0.228673,1.737992,1.430256,0.414340,...,1.640937,0.578033,-0.383420,-0.479271,-0.045596,-0.045596,-0.498831,-0.462818,-0.508550,1.845467
4,-0.359519,0.108454,0.149309,-0.319180,-0.331028,-0.475703,1.110379,-0.778936,-0.822068,0.655420,...,-0.408733,0.709147,0.614951,2.082165,-0.045596,-0.045596,-0.498831,-0.462818,-0.508550,-0.540742


Проверим, что у всех столбцов среднее 0 и среднеквадратичное отклонение 1

In [153]:
dataNormalized.describe()

,age,g,gs,mp,fg,fga,fg.,x3p,x3pa,efg.,...,fta,ft.,orb,pos_C,pos_F,pos_G,pos_PF,pos_PG,pos_SF,pos_SG
count,4.810000e+02,4.810000e+02,4.810000e+02,4.810000e+02,4.810000e+02,4.810000e+02,4.790000e+02,4.810000e+02,4.810000e+02,4.790000e+02,...,4.810000e+02,4.610000e+02,4.810000e+02,4.810000e+02,4.810000e+02,4.810000e+02,4.810000e+02,4.810000e+02,4.810000e+02,4.810000e+02
mean,2.515890e-16,1.521075e-16,7.247610e-17,-3.779605e-17,-2.873654e-17,-7.790026e-17,-6.443466e-16,4.708638e-17,-4.570149e-17,2.284881e-15,...,-1.567743e-17,3.919647e-16,-3.046766e-17,-7.386099e-18,-1.283623e-16,-1.712940e-16,9.324950e-17,5.306451e-16,-2.049643e-16,-9.648092e-17
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-1.788680e+00,-2.063509e+00,-8.622067e-01,-1.377960e+00,-1.122495e+00,-1.150773e+00,-4.423087e+00,-7.789364e-01,-8.296011e-01,-4.829148e+00,...,-9.192458e-01,-4.510432e+00,-8.987076e-01,-4.792708e-01,-4.559608e-02,-4.559608e-02,-4.988309e-01,-4.628177e-01,-5.085504e-01,-5.407417e-01
25%,-8.359060e-01,-8.393113e-01,-8.622067e-01,-9.466462e-01,-8.489736e-01,-8.525496e-01,-3.641986e-01,-7.789364e-01,-8.070025e-01,-2.988544e-01,...,-7.516148e-01,-4.271730e-01,-7.054746e-01,-4.792708e-01,-4.559608e-02,-4.559608e-02,-4.988309e-01,-4.628177e-01,-5.085504e-01,-5.407417e-01
50%,-1.213252e-01,3.059057e-01,-5.250348e-01,-1.074235e-01,-2.728321e-01,-2.506802e-01,1.584714e-02,-4.643203e-01,-4.680239e-01,7.281050e-02,...,-3.630157e-01,1.784481e-01,-3.351113e-01,-4.792708e-01,-4.559608e-02,-4.559608e-02,-4.988309e-01,-4.628177e-01,-5.085504e-01,-5.407417e-01
75%,5.932556e-01,8.982593e-01,9.585214e-01,8.677689e-01,6.641253e-01,6.711016e-01,4.364311e-01,5.581818e-01,6.242406e-01,4.545204e-01,...,4.446610e-01,6.154942e-01,2.767932e-01,-4.792708e-01,-4.559608e-02,-4.559608e-02,-4.988309e-01,-4.628177e-01,-5.085504e-01,-5.407417e-01
max,2.975192e+00,1.174691e+00,1.902603e+00,2.100412e+00,3.818354e+00,3.425603e+00,5.711466e+00,4.353238e+00,3.803107e+00,5.215849e+00,...,5.214525e+00,1.733084e+00,6.186503e+00,2.082165e+00,2.188612e+01,2.188612e+01,2.000520e+00,2.156186e+00,1.962285e+00,1.845467e+00


Выберем игрока, для которого будем искать наиболее близких

In [154]:
selectedPlayer = dataNormalized.loc[ 10 ]
selectedPlayer

age      -0.835906
g         1.056220
gs        1.329410
mp        0.900090
fg        0.239294
fga       0.188522
fg.       0.380691
x3p      -0.523311
x3pa     -0.468024
efg.      0.062766
ft       -0.001985
fta       0.124638
ft.      -0.364738
orb       1.178547
pos_C    -0.479271
pos_F    -0.045596
pos_G    -0.045596
pos_PF   -0.498831
pos_PG   -0.462818
pos_SF    1.962285
pos_SG   -0.540742
Name: 10, dtype: float64

In [155]:
def euclidean_distance( player, selectedPlayer ):
    """
    Считаем "расстояние" между выбранным игроком selected_player и текущим player.
    В нашем случае это обычное евклидово расстояние
    """
      
    dist = 0
    for metricName in predictionColumns:
        dist += ( player[ metricName ] - selectedPlayer[ metricName ] )**2
    
    return dist**0.5

In [156]:
def euclidean_distance2( x, y ):
    """
    Считаем "расстояние" между выбранным игроком selected_player и текущим player.
    Аналогичный расчет для использования в модели KNeighborsRegressor
    """
    
    return np.sum( (x-y)**2 )**0.5

Введем новый столбец "близости" игроков к выбранному

In [157]:
dataNormalized['distance'] = dataNormalized.apply( euclidean_distance, axis = 1, args = (selectedPlayer,) )
dataNormalized.head()

,age,g,gs,mp,fg,fga,fg.,x3p,x3pa,efg.,...,ft.,orb,pos_C,pos_F,pos_G,pos_PF,pos_PG,pos_SF,pos_SG,distance
0,-0.835906,0.384886,-0.862207,-0.435088,-0.738401,-0.768505,0.319884,-0.700282,-0.716608,0.012541,...,-0.389712,0.260690,-0.479271,-0.045596,-0.045596,-0.498831,-0.462818,1.962285,-0.540742,3.248421
1,-1.550487,1.095711,-0.187863,-0.045011,-0.581271,-0.649215,0.674593,-0.778936,-0.829601,0.223485,...,-0.882950,1.387883,2.082165,-0.045596,-0.045596,-0.498831,-0.462818,-0.508550,-0.540742,4.287511
2,0.116868,-0.010016,-0.457600,-0.308035,-0.290291,-0.405214,0.846880,-0.778936,-0.829601,0.394250,...,-0.520826,0.743773,-0.479271,-0.045596,-0.045596,2.000520,-0.462818,-0.508550,-0.540742,4.523142
3,0.355062,0.779789,1.599148,1.465144,1.577804,1.590172,0.228673,1.737992,1.430256,0.414340,...,0.578033,-0.383420,-0.479271,-0.045596,-0.045596,-0.498831,-0.462818,-0.508550,1.845467,5.920274
4,-0.359519,0.108454,0.149309,-0.319180,-0.331028,-0.475703,1.110379,-0.778936,-0.822068,0.655420,...,0.709147,0.614951,2.082165,-0.045596,-0.045596,-0.498831,-0.462818,-0.508550,-0.540742,4.514490


Объединяем вычисления с исходными данными

In [158]:
dataWithDistance = data.join( dataNormalized['distance'] )
dataWithDistance.head()

,player,pos,age,bref_team_id,g,gs,mp,fg,fga,fg.,...,trb,ast,stl,blk,tov,pf,pts,season,season_end,distance
0,Quincy Acy,SF,23,TOT,63,0,847,66,141,0.468,...,216,28,23,26,30,122,171,2013-2014,2013,3.248421
1,Steven Adams,C,20,OKC,81,20,1197,93,185,0.503,...,332,43,40,57,71,203,265,2013-2014,2013,4.287511
2,Jeff Adrien,PF,27,TOT,53,12,961,143,275,0.520,...,306,38,24,36,39,108,362,2013-2014,2013,4.523142
3,Arron Afflalo,SG,28,ORL,73,73,2552,464,1011,0.459,...,262,248,35,3,146,136,1330,2013-2014,2013,5.920274
4,Alexis Ajinca,C,25,NOP,56,30,951,136,249,0.546,...,277,40,23,46,63,187,328,2013-2014,2013,4.514490


In [159]:
dataWithDistance.sort_values( by = 'distance', ascending = True ).head(6)

,player,pos,age,bref_team_id,g,gs,mp,fg,fga,fg.,...,trb,ast,stl,blk,tov,pf,pts,season,season_end,distance
10,Al-Farouq Aminu,SF,23,NOP,80,65,2045,234,494,0.474,...,496,114,82,38,88,147,572,2013-2014,2013,0.000000
251,Michael Kidd-Gilchrist,SF,20,CHA,62,62,1502,167,353,0.473,...,324,52,43,39,60,145,448,2013-2014,2013,1.490506
186,Maurice Harkless,SF,20,ORL,80,41,1950,224,483,0.464,...,264,80,97,50,86,140,592,2013-2014,2013,1.980547
426,Hollis Thompson,SF,22,PHI,77,41,1742,171,372,0.460,...,247,73,53,12,60,144,461,2013-2014,2013,2.171959
16,Giannis Antetokounmpo,SF,19,MIL,77,23,1897,173,418,0.414,...,339,150,60,61,122,173,525,2013-2014,2013,2.295932
267,Kawhi Leonard,SF,22,SAS,66,65,1923,337,645,0.522,...,412,133,114,50,80,127,844,2013-2014,2013,2.379531


Первую строчку не учитываем (это и был selected_player)

Считаем среднее для первых пяти игроков самых близких игроков

In [161]:
dataWithDistance.sort_values( by = 'distance', ascending = True ).iloc[1:6]['pts'].mean()

574.0

## Задание

#### 1. Улучшить точность предсказания.

#### 2. Сделать то же самое с помощью KNeighborsRegressor

In [215]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn import neighbors
from sklearn.model_selection import train_test_split
n_neighbors = 5 
#choosing X and y
X = dataNormalized
y = data['pts']
#checking on empty data
X.isnull().sum()
#filling empty numbers
X['ft.'].fillna(X['ft.'].median(), inplace = True)
X['fg.'].fillna(X['fg.'].median(), inplace = True)
X['efg.'].fillna(X['efg.'].median(), inplace = True)
X['distance'].fillna(X['distance'].median(), inplace = True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = neighbors.KNeighborsRegressor( n_neighbors, weights='distance' , metric = 'minkowski', p = 1 )
model.fit(X_train,y_train)


model.predict(X)



array([3.34365846e+02, 2.65000000e+02, 3.62000000e+02, 1.33000000e+03,
       3.28000000e+02, 9.20000000e+01, 1.60300000e+03, 3.03000000e+02,
       7.01000000e+02, 5.27265459e+02, 5.72000000e+02, 4.43497663e+01,
       4.77000000e+02, 5.64000000e+02, 8.10000000e+02, 2.91289519e+02,
       5.25000000e+02, 2.11200000e+03, 2.23445049e+01, 2.83774504e+02,
       1.10700000e+03, 2.50000000e+01, 4.01000000e+02, 2.80000000e+02,
       9.30000000e+02, 6.34854263e+01, 2.40000000e+02, 2.20000000e+01,
       1.70000000e+02, 1.50000000e+02, 5.89074082e+02, 5.58000000e+02,
       7.38000000e+02, 6.16914409e+02, 1.66000000e+02, 9.11000000e+02,
       2.98000000e+02, 1.07000000e+03, 6.66000000e+02, 1.65338034e+02,
       4.03000000e+02, 1.24900000e+03, 4.36000000e+02, 9.11000000e+02,
       2.17000000e+02, 5.73000000e+02, 3.00000000e+00, 7.20000000e+01,
       2.22000000e+02, 4.97000000e+02, 3.78000000e+02, 8.21000000e+02,
       7.63000000e+02, 5.00000000e+00, 1.20000000e+01, 4.87976540e+02,
      

#### 3. И еще раз с помощью KNeighborsRegressor, передавая метрику как функцию

In [197]:
np.shape(X)

(481, 22)

In [198]:
np.shape(y)

(481,)